# Programming Assignment 1: Descision Trees

Student: Cohen Archbold

Class: CS 460g



In [1]:
import pandas as pd
import numpy as np

# Preparing and Binning Synthetic Data

In [2]:
synthetic1_df = pd.read_csv('PokemonData/synthetic-1.csv')
synthetic2_df = pd.read_csv('PokemonData/synthetic-2.csv')
synthetic3_df = pd.read_csv('PokemonData/synthetic-3.csv')
synthetic4_df = pd.read_csv('PokemonData/synthetic-4.csv')

In [3]:
#Lets take a peak
synthetic1_df.head()

,10.58,-0.055609,1
0,10.8130,0.77631,1
1,8.7405,1.26280,1
2,11.1840,4.37280,1
3,12.2720,-2.39240,1
4,10.8260,2.57780,1


In [4]:
# So it looks like the keys are weird... lets fix them

synthetic1_df.columns = ['x' ,'y', 'z']

synthetic2_df.columns = ['x' ,'y', 'z']

synthetic3_df.columns = ['x' ,'y', 'z']

synthetic4_df.columns = ['x' ,'y', 'z']

synthetic1_df.head()

,x,y,z
0,10.8130,0.77631,1
1,8.7405,1.26280,1
2,11.1840,4.37280,1
3,12.2720,-2.39240,1
4,10.8260,2.57780,1


In [5]:
#Great! Now we need to bin the columns x and y for each dataframe

#Lets make a list of all dataframes

df_list = [synthetic1_df, synthetic2_df, synthetic3_df, synthetic4_df]

quantiles = 4

#Loop through the list and bin each x and y column
for df in df_list:
    df['x'] = pd.qcut(df['x'], q=quantiles)#, labels=['x1','x2','x3','x4'])
    df['y'] = pd.qcut(df['y'], q=quantiles)#, labels=['y1','y2','y3','y4'])

synthetic2_df.head()


,x,y,z
0,"(-2.6, -0.726]","(-1.2859999999999998, 1.027]",1
1,"(-0.726, -0.0656]","(1.027, 2.284]",1
2,"(-2.6, -0.726]","(-1.2859999999999998, 1.027]",1
3,"(-0.0656, 0.579]","(1.027, 2.284]",1
4,"(-0.0656, 0.579]","(1.027, 2.284]",1


# Testing
    Here is where I will test functions like entropy and information gain

In [6]:
# I want to see if we can calculate entropy

import d_trees as dt

print(dt.calculate_entropy(synthetic1_df['z']))

#print(dt.calculate_entropy(synthetic1_df.loc[synthetic1_df['x'] == 'x4']['z']))

synthetic1_df['x'].value_counts()

attributes = ['x','y']
attributes.remove('x')
attributes

0.9999817845368457


['y']

In [7]:
# Here lets test information gain

dt.calculate_info_gain('y','z',synthetic1_df)

synthetic1_df.columns

dt.ID3_algorithm(synthetic1_df, 'z', ['x','y'])
dt.ID3_algorithm(synthetic2_df, 'z', ['x','y'])



Tree Node: Label(0/1 yes/no) -1 Attribute: y

In [8]:
 root = dt.ID3_algorithm(synthetic1_df, 'z', ['x','y'])

print(root)
root.below


Tree Node: Label(0/1 yes/no) -1 Attribute: x


[Tree Node: Label(0/1 yes/no) 1 Attribute: (10.018, 12.996],
 Tree Node: Label(0/1 yes/no) 1 Attribute: (2.87, 10.018],
 Tree Node: Label(0/1 yes/no) 0 Attribute: (-2.969, -0.218],
 Tree Node: Label(0/1 yes/no) 0 Attribute: (-0.218, 2.87]]

# Descision Trees Implementation



In [9]:
tree = dt.D_Tree(synthetic2_df, 'z', ['x','y'])

print(tree)

Root- Tree Node: Label(0/1 yes/no) -1 Attribute: y
	Child- Tree Node: Label(0/1 yes/no) -1 Attribute: (-1.2859999999999998, 1.027]
		GrandChild- Tree Node: Label(0/1 yes/no) 1 Attribute: (-2.6, -0.726]
		GrandChild- Tree Node: Label(0/1 yes/no) 1 Attribute: (-0.726, -0.0656]
		GrandChild- Tree Node: Label(0/1 yes/no) 1 Attribute: (-0.0656, 0.579]
		GrandChild- Tree Node: Label(0/1 yes/no) 1 Attribute: (0.579, 2.774]
	Child- Tree Node: Label(0/1 yes/no) -1 Attribute: (1.027, 2.284]
		GrandChild- Tree Node: Label(0/1 yes/no) 1 Attribute: (-0.726, -0.0656]
		GrandChild- Tree Node: Label(0/1 yes/no) 1 Attribute: (-0.0656, 0.579]
		GrandChild- Tree Node: Label(0/1 yes/no) 1 Attribute: (0.579, 2.774]
		GrandChild- Tree Node: Label(0/1 yes/no) 0 Attribute: (-2.6, -0.726]
	Child- Tree Node: Label(0/1 yes/no) -1 Attribute: (2.284, 3.539]
		GrandChild- Tree Node: Label(0/1 yes/no) 1 Attribute: (0.579, 2.774]
		GrandChild- Tree Node: Label(0/1 yes/no) 0 Attribute: (-0.0656, 0.579]
		GrandChild- T